<a href="https://colab.research.google.com/github/sahaanirbannew/birdname-ner-nlp-demo/blob/main/Finding_Bird_Descriptions_and_Summarises_from_eBird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers sentencepiece sacremoses

In [89]:
import requests
import urllib.request 
from bs4 import BeautifulSoup 
from urllib.request import urlopen  
import urllib.request
import html.parser 
from requests.exceptions import HTTPError
from socket import error as SocketError
from http.cookiejar import CookieJar
import pickle
from urllib.request import build_opener, HTTPCookieProcessor
from transformers import pipeline
from googlesearch import search

In [ ]:
summarizer = pipeline("summarization")

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/roberta-base-squad2"
nlp_qa = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [151]:
def load_commNames():
  try:
    dbfile = open('/content/bird_dict_comName', 'rb')     
    bird_dict_comName = pickle.load(dbfile)
    dbfile.close()
    return bird_dict_comName
  except Exception as e:
    print(str(e))
    
def is_concern(speciesCode, ebird_api_key):
  response_cs = requests.get(f"https://species.birds.cornell.edu/bow/api/v1/auxspecies/"+speciesCode+"?category=conservation_status",
      headers={
        "X-eBirdApiToken": ebird_api_key 
      }
      ) 
  is_concern_ = response_cs.json()[0]["description"]
  return is_concern_ 

def return_html_code(url):
  opener = build_opener(HTTPCookieProcessor())
  response = opener.open(eBird_link)
  html = response.read() 
  return html

def search_speciesCode_by_commName(commName,bird_dict_comName):
  for key in bird_dict_comName:
    if bird_dict_comName[key] == commName.lower():
      return key
  return ""

def search_by_commonName_google(commonName): 
  results = search(commonName, tld="com", num=3, stop=3, pause=2) 
  for result in results: 
    if str(result).find("ebird.org") > -1:
      eBird_link = result
      speciesCode = eBird_link.split("https://ebird.org/species/")[0]

      return speciesCode, eBird_link

def get_mid_tweet_text_process_2(query, cue, bird_description): 
  answer = ask_question_huggingfaceRoberta(query, bird_description)
  if answer['score']>0.068: 
    final_answer = cue + answer['answer']
    final_answer = summarizer(final_answer, max_length=int(len(final_answer)*0.75), min_length=0, do_sample=False)
    return final_answer[0]['summary_text']
  else: return ""

def ask_question_huggingfaceRoberta(user_question, passage):  
  passage = str(passage).replace("**",",")
  QA_input = {
    'question': user_question,
    'context': passage
  }
  ans = nlp_qa(QA_input)  
  return(ans)

In [148]:
commName = input("Enter common name: ").lower().strip()
bird_dict_comName = load_commNames() 
speciesCode = search_speciesCode_by_commName(commName,bird_dict_comName) 

if len(speciesCode) == 0:
  speciesCode, eBird_link = search_by_commonName_google(commName)
else: 
  eBird_link = "https://ebird.org/species/"+speciesCode
botw_link ="https://birdsoftheworld.org/bow/species/"+speciesCode+"/cur/introduction" 
ebird_api_key = "68c6i1pl67vt"

soup = BeautifulSoup(return_html_code(eBird_link), 'html.parser')
mat = soup.find_all("p", {"class": "u-stack-sm"})
bird_description = mat[0].text
#is_concern_ = is_concern(speciesCode, ebird_api_key)

print(commName)


header_tweet = commName + "| Kolkata | 7 Sept 2022" 
length = len(header_tweet)

footer_tweet = "-with @shotsbynavin. More info: "+eBird_link
length = length + len(footer_tweet)

hashtags = "#IndiAves #BBCWildlifePOTD #ThePhotoHour #birdphotography" 
length = length + len(hashtags)
avg_length_word = 3
available_space = 280 - length

len_complete_tweet = 281
while len_complete_tweet > 280:
  avg_length_word = avg_length_word +1 
  app_words = int(available_space/avg_length_word)
  print(app_words)
  mid_tweet = summarizer(bird_description, max_length=app_words, min_length=0, do_sample=False)
  mid_tweet_text = mid_tweet[0]['summary_text'].replace("'","").replace(" .",".").strip() 

  complete_tweet = header_tweet + " " + mid_tweet_text + " " + hashtags + " "+ footer_tweet
  len_complete_tweet = len(complete_tweet)
  print(complete_tweet)
  print(len(complete_tweet))
  print("-----")

Enter common name: emereld dove
emereld dove
30
emereld dove| Kolkata | 7 Sept 2022 Brightly-colored dove of the forest floor with bright green wings, coral-red bill, and ash-gray forehead. Male has #IndiAves #BBCWildlifePOTD #ThePhotoHour #birdphotography -with @shotsbynavin. More info: https://ebird.org/species/emedov2
274
-----


In [149]:
bird_description

'Brightly-colored dove of the forest floor with bright green wings, coral-red bill, and ash-gray forehead. Male has an extensive silver cap which the female lacks. Inhabits various forested habitats from lowlands up into montane areas. Often encountered when flushed from a well-shaded trail. Gives a series of solemn-sounding hiccuping coos: “ti-woo, ti-woo, ti-woo.”'

In [153]:
query_set=[]
query_set.append(["where does this bird get its name from?", "The bird gets its name from "])
query_set.append(["Tell me an interesting this about this bird?", "You know what's interesting abut this bird? it's "])
query_set.append(["Where can we find this bird?", "We can find the bird in "])
query_set.append(["how does the bird look?", "The bird is "])
query_set.append(["Describe the bird?", "The bird is "])



for query in query_set:
  tweet_content = get_mid_tweet_text_process_2(query[0], query[1], bird_description) 
  print(tweet_content)

Your max_length is set to 79, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


 It's bright green wings, coral-red bill, and ash-gray forehead .



Your max_length is set to 21, but you input_length is only 9. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


 The bird is Brightly-colored .


Your max_length is set to 24, but you input_length is only 10. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


 The bird is Brightly-colored dove .


In [162]:
!pip install Sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [158]:
import PegasusForConditionalGeneration
import PegasusTokenizerFast
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

ModuleNotFoundError: ignored

In [163]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

ImportError: ignored